In [26]:
from dotenv import load_dotenv


load_dotenv()

True

In [1]:
from src.agent import Agent

agent = Agent()
agent.viz_graph(ipynb=True)

I0000 00:00:1728316657.378803 10751030 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1728316657.541883 10751030 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import os
import logging
from pathlib import Path
import time

def setup_logging():
    logging.basicConfig(level=logging.INFO,
                       format='%(asctime)s - %(levelname)s - %(message)s')

def validate_image_path(image_path):
    path = Path(image_path)
    if not path.exists():
        raise FileNotFoundError(f"Image file not found: {image_path}")
    if not path.is_file():
        raise ValueError(f"Path is not a file: {image_path}")
    if path.suffix.lower() not in ['.jpg', '.jpeg', '.png', '.gif', '.bmp']:
        raise ValueError(f"Unsupported file type: {path.suffix}")
    return str(path.absolute())

def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920x1080')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    
    try:
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=options
        )
        return driver
    except Exception as e:
        logging.error(f"Failed to create driver: {str(e)}")
        raise

def search_image(image_path):
    setup_logging()
    
    try:
        # Validate image path
        validated_path = validate_image_path(image_path)
        logging.info(f"Validated image path: {validated_path}")
        
        # Create driver
        driver = create_driver()
        wait = WebDriverWait(driver, 20)
        
        try:
            # Navigate to Google Images
            driver.get("https://images.google.com")
            logging.info("Navigated to Google Images")
            
            # Click camera icon
            search_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//div[@aria-label="Search by image"]'))
            )
            search_button.click()
            
            # Click upload tab
            upload_tab = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//a[text()="Upload an image"]'))
            )
            upload_tab.click()
            
            # Upload image
            upload_input = wait.until(
                EC.presence_of_element_located((By.XPATH, '//input[@type="file"]'))
            )
            upload_input.send_keys(validated_path)
            logging.info("Uploaded image successfully")
            
            # Wait for results and extract them
            # Wait for any element that indicates results have loaded
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.g'))
            )
            
            # Get results
            results = []
            elements = driver.find_elements(By.CSS_SELECTOR, 'div.g a')
            for element in elements[:5]:
                try:
                    url = element.get_attribute('href')
                    if url and url.startswith('http'):
                        results.append(url)
                except Exception as e:
                    logging.warning(f"Failed to extract URL: {str(e)}")
                    
            return results
            
        finally:
            driver.quit()
            logging.info("Browser closed")
            
    except Exception as e:
        logging.error(f"Search failed: {str(e)}")
        raise

try:
    # Replace with your image path
    image_path = '1.jpg'
    results = search_image(image_path)
    
    print("\nSearch Results:")
    for i, url in enumerate(results, 1):
        print(f"{i}. {url}")
        
except Exception as e:
    print(f"Error: {str(e)}")



2024-10-08 13:02:27,344 - INFO - Validated image path: /workspaces/GeoAI/test/test_images/test.jpg
2024-10-08 13:02:27,346 - INFO - ====== WebDriver manager ======
2024-10-08 13:02:27,365 - INFO - Get LATEST chromedriver version for google-chrome
2024-10-08 13:02:27,389 - INFO - About to download new driver from https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
2024-10-08 13:02:27,409 - INFO - Driver downloading response is 200


2024-10-08 13:02:27,486 - INFO - Get LATEST chromedriver version for google-chrome
2024-10-08 13:02:27,667 - INFO - Get LATEST chromedriver version for google-chrome
2024-10-08 13:02:27,696 - INFO - Driver has been saved in cache [/home/codespace/.wdm/drivers/chromedriver/linux64/114.0.5735.90]
2024-10-08 13:02:27,789 - ERROR - Failed to create driver: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5b2f43c614e3 <unknown>
#1 0x5b2f43990c76 <unknown>
#2 0x5b2f439b7757 <unknown>
#3 0x5b2f439b6029 <unknown>
#4 0x5b2f439f4ccc <unknown>
#5 0x5b2f439f447f <unknown>
#6 0x5b2f439ebde3 <unknown>
#7 0x5b2f439c12dd <unknown>
#8 0x5b2f439c234e <unknown>
#9 0x5b2f43c213e4 <unknown>
#10 0x5b2f43c253d7 <unknown>
#11 0x5b2f43c2fb20 <unknown>
#12 0x5b2f43c26023 <unknown>
#13 0x5b2f43bf41aa <unknown>
#14 0x5b2f43c4a6b8 <unknown>
#15 0x5b2f43c4a847 <unknown>
#16 0x5b2f43c5a243 <unknown>
#17 0x758593e63609 start_thread

2024-10-08 13:02:27,790 - ERROR - Search failed: Message: unknown e

Error: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5b2f43c614e3 <unknown>
#1 0x5b2f43990c76 <unknown>
#2 0x5b2f439b7757 <unknown>
#3 0x5b2f439b6029 <unknown>
#4 0x5b2f439f4ccc <unknown>
#5 0x5b2f439f447f <unknown>
#6 0x5b2f439ebde3 <unknown>
#7 0x5b2f439c12dd <unknown>
#8 0x5b2f439c234e <unknown>
#9 0x5b2f43c213e4 <unknown>
#10 0x5b2f43c253d7 <unknown>
#11 0x5b2f43c2fb20 <unknown>
#12 0x5b2f43c26023 <unknown>
#13 0x5b2f43bf41aa <unknown>
#14 0x5b2f43c4a6b8 <unknown>
#15 0x5b2f43c4a847 <unknown>
#16 0x5b2f43c5a243 <unknown>
#17 0x758593e63609 start_thread

